In [121]:
%%capture
try:
    from pip import main as pipmain
except:
    from pip._internal import main as pipmain
packages = ['twython', 'pandas', 'psycopg2-binary']
pipmain(['install'] + packages)

In [122]:
%%capture
from twython import Twython

from nltk.tokenize import sent_tokenize, word_tokenize, casual_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

import re
import random
import pandas as pd
import psycopg2

In [123]:
import sys
sys.path.append('../')
from auth import (
    consumer_key,
    consumer_secret,
    access_token,
    access_token_secret
)

twitter = Twython(
    consumer_key,
    consumer_secret,
    access_token,
    access_token_secret
)
from aws import host, port, user, password, database

connection = psycopg2.connect(host = host, port = port, user = user, password = password, dbname = database)
cursor=connection.cursor()

In [194]:
def read_try(sql):
    try:
        df = pd.read_sql(sql, con=connection)
        return pd.DataFrame() if df.empty else df
    except Exception as e:
        #print("READ ERROR", e)
        return pd.DataFrame()

def write_try(sql):
    try:
        cursor.execute(sql)  # run a psql command
        return True
    except Exception as e:
        #print("WRITE ERROR: ", e)
        return False
    finally:
        connection.commit()
        
# get all the users from the users table
def read_all_users_from_db():
    sql = 'SELECT * FROM users'
    return read_try(sql)

# get all the follower, user pairs from the followers table
def read_all_followers_from_db():
    sql = 'SELECT * FROM followers'
    return read_try(sql)

# get all the tweets from tweets table
def read_all_tweets_from_db():
    sql = 'SELECT * FROM tweets'
    return read_try(sql)

# get all word counts of tweet text
def read_all_words_from_db():
    sql = 'SELECT * FROM words'
    return read_try(sql)

# get all hashtag counts
def read_all_hashtags_from_db():
    sql = 'SELECT * FROM hashtags'
    return read_try(sql)

def read_user_from_db(user):
    sql = 'SELECT * FROM users WHERE user_handle = \'{}\''.format(user)
    return read_try(sql)

# get list of followers for a user
def read_user_followers_from_db(user, limit = 'NULL'):
    sql = 'SELECT follower_handle FROM followers WHERE user_handle = \'{}\'LIMIT {}'.format(user, limit)
    return read_try(sql)

# see if tweet is already indexed in DB
def read_tweet_from_db(tweet_id):
    sql = ('SELECT tweet_date FROM tweets '
    'WHERE tweet_id = {}'.format(tweet_id)
    )
    return read_try(sql)

# get list of word counts for a user
def read_user_words_from_db(user, limit = 'NULL', date = ''):
    sql = ('SELECT word, SUM(count) as sum_count FROM followers '
    'JOIN tweets ON tweets.follower_handle = followers.follower_handle '
    'JOIN words ON words.tweet_id = tweets.tweet_id '
    'WHERE followers.user_handle = \'{}\' GROUP BY word ORDER BY sum_count desc LIMIT {}'.format(user, limit)
    )
    return read_try(sql)

# get list of hashtag counts for a user
def read_user_hashtags_from_db(user, limit = 'NULL', date = ''):
    sql = ('SELECT word, SUM(count) as sum_count FROM followers '
    'JOIN tweets ON tweets.follower_handle = followers.follower_handle '
    'JOIN hashtags ON hashtags.tweet_id = tweets.tweet_id '
    'WHERE followers.user_handle = \'{}\' GROUP BY word ORDER BY sum_count desc LIMIT {}'.format(user, limit)
    )
    return read_try(sql)

def write_user_to_db(user):
    sql = 'INSERT INTO users VALUES (\'{}\');'.format(user)
    write_try(sql)

def write_user_followers_to_db(followers, user):
    for follower in followers:
        sql = 'INSERT INTO followers VALUES (\'{}\', \'{}\');'.format(follower, user)
        write_try(sql)

In [127]:
PRINT_LIMIT = 50
FOLLOWER_SAMPLE_LIMIT = 20 # number of followers to randomly sample
WORD_FREQ_LIMIT = 10 # return this number of topics that are most freq

In [128]:
user = "AndrewYang"

In [129]:
followers = twitter.get_followers_list(screen_name = user)['users']
print(followers)

[{'id': 126116060, 'id_str': '126116060', 'name': 'Phi Phi Gregs', 'screen_name': 'alphapandora', 'location': 'Detroit ', 'description': "Who knew the only way to get accurate and up to the minute news in this day and age would be Twitter? I'm in for it sis.", 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 63, 'friends_count': 76, 'listed_count': 3, 'created_at': 'Wed Mar 24 22:00:17 +0000 2010', 'favourites_count': 27753, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': False, 'statuses_count': 9092, 'lang': None, 'status': {'created_at': 'Sun Nov 24 01:15:05 +0000 2019', 'id': 1198409672073662465, 'id_str': '1198409672073662465', 'text': 'RT @AndrewYang: Was asked to appear on @msnbc this weekend - and told them that I’d be happy to after they apologize on-air, discuss and in…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'AndrewYang', 'name': 'Andrew Yang🧢', '

In [130]:
limit = 0
followers_list = []
for follower in followers:
    followers_list.append(follower['screen_name'])
print(followers_list)

['alphapandora', 'rcole1954', 'I_dont_tweet11', 'bigwindfarm', 'snarkykat', 'GraemeRich', '_lolabrowne', 'TrachtenbergAli', 'j_jvsmine', 'ShannonCondon5', 'fletcherstories', 'MJ_Adamson', 'MAKO1954', 'uziwalker_', 'yedispaghetti', 'jojobecat2', 'HuchChow', 'DavidAcquaah', 'rayndeigh', 'ACHU_555']


In [131]:
# add user to users db
write_user_to_db(user)
read_all_users_from_db()

WRITE ERROR:  duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_handle)=(AndrewYang) already exists.



,user_handle
0,test_user
1,test_user1
2,realDonaldTrump
3,AndrewYang


In [132]:
# add followers followers db
write_user_followers_to_db(followers_list, user)
read_all_followers_from_db()

,follower_handle,user_handle
0,test_follower1,test_user
1,test_follower2,test_user
2,test_follower1,test_user1
3,james_graziano,AndrewYang
4,rbrsq,AndrewYang
5,BrettLipkin,AndrewYang
6,ROCKYMTNBOODA,AndrewYang
7,KaylinEHill,AndrewYang
8,PackerK_U,AndrewYang
9,dante_lentz,AndrewYang


In [133]:
def get_tweets_from(follower_name, results = []):
    try: 
        result = twitter.get_user_timeline(screen_name = follower_name)
        results.extend(result)
        return results
    except:
        return results

In [134]:
def get_tweet_info_from_tweets(results):
    tweets = []
    for tweet in results:
        tweet_map = {'text': tweet['text']}
        tweet_map['id'] = tweet['id']
        tweet_map['date'] = tweet['created_at']
        tweet_map['user'] = tweet['user']['screen_name']
        tweet_hashtags = []
        tweet_urls = []
        tweet_user_mentions = []
        
        entities = tweet['entities']
    
        for hashtags in entities['hashtags']:
            tweet_hashtags.append(hashtags['text'])
        for urls in entities['urls']:
            tweet_urls.append(urls['url'])
        try:
            for media in entities['media']:
                tweet_urls.append(media['url'])
        except:
            pass
        for users in entities['user_mentions']:
            tweet_user_mentions.append(users['screen_name'])
            
        tweet_map["hashtags"] = tweet_hashtags
        tweet_map["urls"] = tweet_urls
        tweet_map["user_mentions"] = tweet_user_mentions
        tweets.append(tweet_map)
        
    return tweets

In [135]:
results = []
for f in followers_list:
    results = get_tweets_from(f, results)
print(results)

[{'created_at': 'Sun Nov 24 01:15:27 +0000 2019', 'id': 1198409761353543680, 'id_str': '1198409761353543680', 'text': 'RT @AOC: Activism disrupts the present to change the future. 🌎 https://t.co/K4OLyeATMu', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'AOC', 'name': 'Alexandria Ocasio-Cortez', 'id': 138203134, 'id_str': '138203134', 'indices': [3, 7]}], 'urls': [{'url': 'https://t.co/K4OLyeATMu', 'expanded_url': 'https://twitter.com/sinow/status/1198323039068401666', 'display_url': 'twitter.com/sinow/status/1…', 'indices': [63, 86]}]}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 126116060, 'id_str': '126116060', 'name': 'Phi Phi Gregs', 'screen_name': 'alphapandora', 'location': 'Detroit ', 'description': "Wh

In [136]:
tweets_data = get_tweet_info_from_tweets(results)
tweets_data

[{'text': 'RT @AOC: Activism disrupts the present to change the future. 🌎 https://t.co/K4OLyeATMu',
  'id': 1198409761353543680,
  'date': 'Sun Nov 24 01:15:27 +0000 2019',
  'user': 'alphapandora',
  'hashtags': [],
  'urls': ['https://t.co/K4OLyeATMu'],
  'user_mentions': ['AOC']},
 {'text': 'RT @AndrewYang: Was asked to appear on @msnbc this weekend - and told them that I’d be happy to after they apologize on-air, discuss and in…',
  'id': 1198409672073662465,
  'date': 'Sun Nov 24 01:15:05 +0000 2019',
  'user': 'alphapandora',
  'hashtags': [],
  'urls': [],
  'user_mentions': ['AndrewYang', 'MSNBC']},
 {'text': 'RT @KamalaHarris: Vice President Mike Pence, Secretary of State Mike Pompeo, and Chief of Staff Mick Mulvaney are part of a criminal enterp…',
  'id': 1198409625105817600,
  'date': 'Sun Nov 24 01:14:54 +0000 2019',
  'user': 'alphapandora',
  'hashtags': [],
  'urls': [],
  'user_mentions': ['KamalaHarris']},
 {'text': "RT @DanRather: A horrific environmental disaster in

[NLTK Stopwords](https://pythonspot.com/nltk-stop-words/)

In [137]:
STOPWORDS = stopwords.words('english') + stopwords.words('spanish')
# TODO: decide whether or not to remove tweets that are from retweets altogether
more_stop_words = ['rt']
STOPWORDS += more_stop_words
STOPWORDS

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

POS tag each word using NLTK   
Resources:
- [Categorizing and Tagging Words](https://www.nltk.org/book/ch05.html)
- [NLTK tags](https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/)

Noun tags:
- NN noun, singular 'desk'
- NNS noun plural 'desks'
- NNP proper noun, singular 'Harrison'
- NNPS proper noun, plural 'Americans'

In [199]:
# noun based word extraction
def word_hashtag_extraction_noun_based(tweet_dictionary, stopwords = STOPWORDS):  
    text = casual_tokenize(tweet_dictionary['text'])
    result = nltk.pos_tag(text)
    
    hashtags = ['#' + s for s in tweet_dictionary['hashtags']]
    urls = tweet_dictionary['urls']
    user_mentions = ['@' + s for s in tweet_dictionary['user_mentions']]
    ignore = hashtags + urls + user_mentions
    
    noun_tags = ['NN', 'NNS', 'NNP', 'NNPS']

    cleaned_text = []
    for t in result:
        word = t[0]
        tag = t[1]
        if (tag in noun_tags) and (not word in ignore) and (not word.lower() in stopwords) and (word.isalpha()):
            cleaned_text.append(word.lower())
            
    return cleaned_text, set([h.lower() for h in hashtags])

In [139]:
test_tweet = tweets_data[0]
print(test_tweet)
words, tags = word_hashtag_extraction_noun_based(test_tweet)
print(words)
print(tags)

{'text': 'RT @AOC: Activism disrupts the present to change the future. 🌎 https://t.co/K4OLyeATMu', 'id': 1198409761353543680, 'date': 'Sun Nov 24 01:15:27 +0000 2019', 'user': 'alphapandora', 'hashtags': [], 'urls': ['https://t.co/K4OLyeATMu'], 'user_mentions': ['AOC']}
['activism', 'future']
set()


In [196]:
def get_freq_map(my_list): 
    freq = {} 
    for item in my_list: 
        if (item in freq): 
            freq[item] += 1
        else: 
            freq[item] = 1
    return freq

In [141]:
word_count = get_freq_map(words)
print(word_count)
hashtag_count = get_freq_map(tags)
print(hashtag_count)

{'activism': 1, 'future': 1}
{}


In [197]:
for w,c in word_count.items():
    print(w,c)

activism 1
future 1


In [202]:
def write_tweets_words_hashtags_to_db(tweets_dictionary):
    for tweet in tweets_dictionary:
        follower = tweet['user']
        tweet_id = tweet['id']
        tweet_date = tweet['date']
        sql = 'INSERT INTO tweets VALUES ({},\'{}\',\'{}\');'.format(tweet_id, follower, tweet_date)

        # if writing tweet to database was sucessful, then we need to get write word count info
        if write_try(sql):
            pass
            words, tags = word_hashtag_extraction_noun_based(tweet)
            word_count = get_freq_map(words)
            hashtag_count = get_freq_map(tags)
            
            for w, c in word_count.items():
                sql2 = 'INSERT INTO words VALUES ({},\'{}\',{});'.format(tweet_id, w, c)
                write_try(sql2)

            for w, c in hashtag_count.items():
                sql2 = 'INSERT INTO hashtags VALUES ({},\'{}\',{});'.format(tweet_id, w, c)
                write_try(sql2)

In [171]:
[tweets_data[0]]

[{'text': 'RT @AOC: Activism disrupts the present to change the future. 🌎 https://t.co/K4OLyeATMu',
  'id': 1198409761353543680,
  'date': 'Sun Nov 24 01:15:27 +0000 2019',
  'user': 'alphapandora',
  'hashtags': [],
  'urls': ['https://t.co/K4OLyeATMu'],
  'user_mentions': ['AOC']}]

In [201]:
write_tweets_words_hashtags_to_db(tweets_data)

In [203]:
all_tweets = read_all_tweets_from_db()
print(all_tweets)
all_words = read_all_words_from_db()
print(all_words)
all_hashtags = read_all_hashtags_from_db()
print(all_hashtags)

                tweet_id follower_handle          tweet_date
0                      0  test_follower1 1977-01-08 04:05:06
1                      1  test_follower1 1999-01-08 04:05:06
2    1198409761353543680    alphapandora 2019-11-24 01:15:27
3    1198409672073662465    alphapandora 2019-11-24 01:15:05
4    1198409625105817600    alphapandora 2019-11-24 01:14:54
..                   ...             ...                 ...
264  1197321346709102592       rayndeigh 2019-11-21 01:10:28
265  1197320574172901376       rayndeigh 2019-11-21 01:07:24
266  1197318287622885376       rayndeigh 2019-11-21 00:58:19
267  1197302890160934912       rayndeigh 2019-11-20 23:57:08
268  1196636691316051968       rayndeigh 2019-11-19 03:49:54

[269 rows x 3 columns]
                tweet_id      word  count
0                      0    cheese     45
1                      0  thankyou      3
2                      1    lovely      2
3                      1    cheese      5
4                      0     apple

In [207]:
read_user_words_from_db(user)

,word,sum_count
0,people,10
1,time,6
2,money,6
3,australia,6
4,someone,6
...,...,...
763,dignitaries,1
764,artifact,1
765,investment,1
766,democrats,1


In [208]:
read_user_words_from_db(user, limit = WORD_FREQ_LIMIT)

,word,sum_count
0,people,10
1,australia,6
2,today,6
3,money,6
4,someone,6
5,trump,6
6,time,6
7,game,5
8,years,5
9,please,5


In [209]:
read_user_hashtags_from_db(user, limit = WORD_FREQ_LIMIT)

,word,sum_count
0,#personhood,6
1,#obesity,4
2,#wmtgreen,3
3,#weightloss,2
4,#sustainability,2
5,#health,2
6,#walmart,2
7,#joymatters,2
8,#bariatrics,1
9,#bam,1
